In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:
review_json_path = 'Books_5.json'

size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      chunksize=size)



In [3]:
for chunk_review in review:
    sample_list = pd.DataFrame(chunk_review[['reviewerID','asin','overall']])
    
    print('run')


run


KeyboardInterrupt: 

In [4]:
chunk_review.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,False,"03 30, 2005",A1REUF3A1YCPHM,0001713353,{'Format:': ' Hardcover'},TW Ervin II,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,1112140800,NaN,NaN
1,5,True,"06 20, 2016",AVP0HXC9FG790,0001713353,NaN,Amazon Customer,The kids loved it!,Five Stars,1466380800,NaN,NaN
2,5,True,"01 24, 2016",A324TTUBKTN73A,0001713353,{'Format:': ' Paperback'},Tekla Borner,My students (3 & 4 year olds) loved this book!...,Five Stars,1453593600,NaN,NaN
3,5,False,"07 9, 2015",A2RE7WG349NV5D,0001713353,{'Format:': ' Paperback'},Deborah K Woroniecki,LOVE IT,Five Stars,1436400000,NaN,NaN
4,5,True,"01 18, 2015",A32B7QIUDQCD0E,0001713353,NaN,E,Great!,Five Stars,1421539200,NaN,NaN


In [5]:
book_count = (sample_list.
     groupby(by = ['asin'])['overall'].
     count().
     reset_index()
    )


threshold = 50
book_count = book_count.query('overall >= @threshold')

book_count.head()

,asin,overall
3,000100039X,772
4,0001046314,368
7,0001047868,223
8,0001048767,3018
10,000105001X,114


In [6]:
user_rating = pd.merge(book_count, sample_list, left_on='asin', right_on='asin', how='left')

user_count = (sample_list.
     groupby(by = ['reviewerID'])['overall'].
     count().
     reset_index()
    )

threshold = 20
user_count = user_count.query('overall >= @threshold')

user_count.head()

,reviewerID,overall
194,A100YDQ9DIP06P,23
405,A102Z3T7NSM5KC,20
747,A106016KSI0YQ,22
1189,A10A1S5NAQBT21,22
1405,A10BZSGALQPS0V,23


In [7]:
combined = user_rating.merge(user_count, left_on = 'reviewerID', right_on = 'reviewerID', how = 'inner')

combined.head()

,asin,overall_x,reviewerID,overall_y,overall
0,000100039X,772,A2UIILZDC35P15,5,27
1,0001048767,3018,A2UIILZDC35P15,5,27
2,0001050230,2228,A2UIILZDC35P15,5,27
3,0007107005,249,A2UIILZDC35P15,4,27
4,0007155662,4575,A2UIILZDC35P15,3,27


In [8]:
print('Number of unique books: ', combined['asin'].nunique())


Number of unique books:  3233


In [9]:
print('Number of unique users: ', combined['reviewerID'].nunique())


Number of unique users:  1913


In [10]:
combined_gb = combined.groupby(by = ['reviewerID','asin'],as_index=False).mean()

combined_final = combined_gb.drop(columns = ['overall_x','overall'])
combined_final = combined_final.rename(columns = {'reviewerID':'User-ID', 'asin': 'Book-ID', 'overall_y':'Rating'})
scaler = MinMaxScaler()
combined_final['Rating'] = combined_final['Rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined_final['Rating'].values.reshape(-1,1)))
combined_final['Rating'] = rating_scaled

combined_final.head()

,User-ID,Book-ID,Rating
0,A100YDQ9DIP06P,0001047868,1.00
1,A100YDQ9DIP06P,0060219106,1.00
2,A100YDQ9DIP06P,0060232943,0.75
3,A100YDQ9DIP06P,0060766212,1.00
4,A100YDQ9DIP06P,0060783338,0.75


In [11]:
combined = combined_final.drop_duplicates(['User-ID', 'Book-ID'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-ID', values='Rating')
user_book_matrix.fillna(0, inplace=True)
users = user_book_matrix.index.tolist()
books = user_book_matrix.columns.tolist()


In [12]:
user_book_matrix

Book-ID,000100039X,0001046314,0001047868,0001048767,000105001X,0001050230,0001052292,0001381733,0001384198,0001712713,...,0061703796,0061706515,006170654X,0061706558,0061706620,0061708712,0061708771,0061709697,0061710245,0061713244
User-ID,,,,,,,,,,,,,,,,,,,,,
A100YDQ9DIP06P,0.0,0.0,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A102Z3T7NSM5KC,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A106016KSI0YQ,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10A1S5NAQBT21,0.0,0.0,0.0,1.0,0.0,1.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10BZSGALQPS0V,0.0,0.0,0.0,0.0,0.0,0.0,0.75,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZFX7FLEHCWXM,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0
AZHOZFC4VWIMF,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZK12FBX7X7LN,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
combined_final['Rating'].value_counts()


1.000000    24749
0.750000    17161
0.500000     7486
0.250000     2480
0.000000      989
0.875000      292
0.625000      139
0.916667       49
0.833333       47
0.375000       29
0.937500       19
0.687500       11
0.666667       10
0.812500        8
0.950000        6
0.850000        6
0.583333        5
0.900000        4
0.125000        4
0.333333        3
0.906250        3
0.791667        3
0.892857        3
0.972222        2
0.166667        2
0.964286        2
0.954545        2
0.416667        2
0.944444        2
0.437500        1
0.083333        1
0.865385        1
0.428571        1
0.659091        1
0.956522        1
0.967391        1
0.968750        1
0.983333        1
0.334821        1
0.982143        1
0.818182        1
0.785714        1
0.980769        1
0.526316        1
0.961538        1
0.821429        1
0.769231        1
0.928571        1
0.562500        1
0.936747        1
0.694444        1
0.958333        1
0.681818        1
Name: Rating, dtype: int64